In [ ]:
# Install necessary packages
!pip install tensorflow==2.12.0
!pip install efficientnet
!pip install gradio opencv-python

In [ ]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.12.0
2.12.0


# **GUI is created with Gradio**

In [ ]:
#Import libraries
import gradio as gr
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#I add
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

# Load the segmentation model
segmentation_model = load_model('/content/drive/MyDrive/Colab Notebooks/Project Implementation/New_Experiments/Saved_model/Segmentation_UNET_Resnet34_ORIGA&RIMONEDL_with_Aug_V2_2040_without_denoising_Epoch_50_batch_16.hdf5', compile=False)

# Load the classification model
classification_model = load_model('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/Saved_model_RIM-ONE_DL/After_use_all_augmented_for_Train/Classification_EfficientNetB0_Aug_Train_RIM-ONE_DL_epoch_15_batch_8_lr=0.0001_version2_model.h5')
#
# Define the preprocessing function for the segmentation model
def preprocess_segmentation_image(image):
    # Apply resize
    new_width = 128
    new_height = 128
    dim = (new_width, new_height)
    resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    # Preprocess the image
    preprocessed_image = resized_image / 255.
    return preprocessed_image

# Define the preprocessing function for the classification model
def preprocess_classification_image(image):
    # Resize the image
    img = cv2.resize(image, (224, 224))
    # Normalize the image
    img = img.astype('float32') / 255
    # Reshape the image to match the model's input shape
    img = np.expand_dims(img, axis=0)
    return img

def predict(image):
    # Convert the image from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Preprocess the image for segmentation
    preprocessed_image = preprocess_segmentation_image(image)

    # Make segmentation prediction
    segmentation_prediction = segmentation_model.predict(np.expand_dims(preprocessed_image, axis=0))
    segmentation_mask = np.argmax(segmentation_prediction, axis=3)[0]

    # Resize the mask to match the image size
    segmentation_mask_resized = cv2.resize(segmentation_mask.astype(np.uint8), (image.shape[1], image.shape[0]))

    # Apply the mask to the original image
    segmented_image = cv2.bitwise_and(image, image, mask=segmentation_mask_resized)

    # Convert the segmented image to grayscale
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)

    # Find contours in the grayscale image
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the bounding rectangle for each contour
    bounding_rectangles = [cv2.boundingRect(contour) for contour in contours]

    # Get the x, y, w, h of the bounding rectangle with the maximum area
    x, y, w, h = max(bounding_rectangles, key=lambda rect: rect[2] * rect[3])

    # Crop the black area from the segmented image
    cropped_segmented_image = segmented_image[y:y+h, x:x+w]

    # Preprocess the cropped segmented image for classification
    preprocessed_segmented_image = preprocess_classification_image(cropped_segmented_image)

    # Make classification prediction
    classification_prediction = classification_model.predict(preprocessed_segmented_image)
    label = "Glaucoma" if classification_prediction[0][0] >= 0.738232 else "Normal" #rather than 0.5

    # Convert the cropped segmented image back to RGB
    cropped_segmented_image = cv2.cvtColor(cropped_segmented_image, cv2.COLOR_BGR2RGB)

    return cropped_segmented_image, label


iface = gr.Interface(fn=predict, inputs="image", outputs=["image", "text"], title="Glaucoma Detection on Enhanced Retinal Images Based Deep Learning")
iface.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9896a22aa8b1d56fe9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
